In [119]:
#import pyvips   # create image pyramid from tiles
from openslide import open_slide
from openslide.deepzoom import DeepZoomGenerator
from PIL import Image
import cv2
import numpy as np
from matplotlib import pyplot as plt
import tifffile as tiff   # we want to write/handle our images as "TIFF"
import glob
import os

In [121]:
### Save normalized image tiles at all levels for all patient WSIs
# We also want to sort the images into "Strong_signal" and "Weak_signal"

# Get list of all patient WSIs
dir_name = "/Users/anirudhgangadhar/Desktop/UHN_Postdoc/Datasets/Liver_biopsy_patient_images/"
wsi_img_list=(glob.glob(dir_name + "*.svs"))

# Change path to wherever you want to store the image tiles
os.chdir('/Users/anirudhgangadhar/Desktop/UHN_Postdoc/Data and results/Liver_biopsy/Patient image sets/')
for i in range(len(wsi_img_list)): 
    
    # if we want to test for 1 WSI case 
#     if i == 1:
#         break
    
    # Get WSI filename w/o extension - we will use this to name folders
    filename_noext = os.path.splitext(os.path.basename(wsi_img_list[i]))[0]
    
    # Create unique folder for patient
    parent_dir = '/Users/anirudhgangadhar/Desktop/UHN_Postdoc/Data and results/Liver_biopsy/Patient image sets/'
    directory = filename_noext
    path = os.path.join(parent_dir, directory)
    
    # check if directory exists already - if so don't create new directory
    if os.path.exists(path) == False:
        
        print(f'Saving WSI: {filename_noext} for quantitative analysis ...')
        os.mkdir(path)
    
        # Create sub-folders for different tile levels
        # Read WSI image
        slide = open_slide(wsi_img_list[i])
    
        tile_sz = 256
        tiles = DeepZoomGenerator(slide, tile_size=tile_sz, overlap=0, limit_bounds=False)
        num_levels = tiles.level_count
        
        # Create folder for each tile level
        for j in range(15):  # All levels will generate too may images
            cols, rows = tiles.level_tiles[j]
            # First, create unique folder for each tile level
            path_level = os.path.join(path, '%d' % j)
            os.mkdir(path_level)
            
            # create sub-folders for segregating tile images
            parent_dir_level = path_level
            directory_level_1, directory_level_2 = 'High_signal', 'Low_signal'
            path_level_seg_1 = os.path.join(parent_dir_level, directory_level_1)
            path_level_seg_2 = os.path.join(parent_dir_level, directory_level_2)
            os.mkdir(path_level_seg_1)
            os.mkdir(path_level_seg_2)
            
            # Saving image tiles at each level
            for row in range(rows):
                for col in range(cols):
                    tile_name = os.path.join(path_level, '%s_%d_%d' % (filename_noext, col, row))
                    temp_tile = tiles.get_tile(j, (col, row))
                    temp_tile_np = np.array(temp_tile.convert('RGB'))
                    plt.imsave(tile_name + ".tiff", temp_tile_np)
                    
#                     image = tiff.imread(tile_name)
                    # use SD to filter
                    if (temp_tile_np.std() >= 10):
                        tile_name_1 = os.path.join(path_level_seg_1, '%s_%d_%d' % (filename_noext, col, row))
                        plt.imsave(tile_name_1 + ".tiff", temp_tile_np)
                    else:
                        tile_name_2 = os.path.join(path_level_seg_2, '%s_%d_%d' % (filename_noext, col, row))
                        plt.imsave(tile_name_2 + ".tiff", temp_tile_np)
        print(f'WSI: {filename_noext} imageset created !')
    else:
        print('Folder already exists !!')

Saving images for quantitative analysis ...
WSI imageset created !
Saving images for quantitative analysis ...
WSI imageset created !
Saving images for quantitative analysis ...
WSI imageset created !
Saving images for quantitative analysis ...
WSI imageset created !
Saving images for quantitative analysis ...
WSI imageset created !
Saving images for quantitative analysis ...
WSI imageset created !
Saving images for quantitative analysis ...
WSI imageset created !
Saving images for quantitative analysis ...
WSI imageset created !
Saving images for quantitative analysis ...
WSI imageset created !
Saving images for quantitative analysis ...
WSI imageset created !
Saving images for quantitative analysis ...
WSI imageset created !
Saving images for quantitative analysis ...
WSI imageset created !
Saving images for quantitative analysis ...
WSI imageset created !
Saving images for quantitative analysis ...
WSI imageset created !
Saving images for quantitative analysis ...
WSI imageset creat

In [104]:
# # ### Function to normalize HnE stained images - is this needed for DL ?
# # # NOTE: does not work if image is blank

# # https://youtu.be/tNfcvgPKgyU

# # Original video on H&E topic: https://youtu.be/yUrwEYgZUsA

# # Credit: https://github.com/bnsreenu/python_for_microscopists/blob/master/267_processing_whole_slide_images/normalize_HnE.py

# '''
# This code normalizes staining appearance of H&E stained images.
# It also separates the hematoxylin and eosing stains in to different images. 
# Workflow based on the following papers:
# A method for normalizing histology slides for quantitative analysis. 
# M. Macenko et al., ISBI 2009
#     http://wwwx.cs.unc.edu/~mn/sites/default/files/macenko2009.pdf
# Efficient nucleus detector in histopathology images. J.P. Vink et al., J Microscopy, 2013
# Original MATLAB code:
#     https://github.com/mitkovetta/staining-normalization/blob/master/normalizeStaining.m
 
# Other useful references:
#     https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5226799/
#     https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0169875
# PROPOSED WORKFLOW:  
    
# Input: RGB image
# Step 1: Convert RGB to OD (optical density)
# Step 2: Remove data with OD intensity less than β
# Step 3: Calculate  singular value decomposition (SVD) on the OD tuples
# Step 4: Create plane from the SVD directions corresponding to the
# two largest singular values
# Step 5: Project data onto the plane, and normalize to unit length
# Step 6: Calculate angle of each point wrt the first SVD direction
# Step 7: Find robust extremes (αth and (100−α)th 7 percentiles) of the
# angle
# Step 8: Convert extreme values back to OD space
# Output: Optimal Stain Vectors
# '''

# ############### INPUT RGB IMAGE #######################
# #Using opencv to read images may bemore robust compared to using skimage
# #but need to remember to convert BGR to RGB.
# #Also, convert to float later on and normalize to between 0 and 1.

# #Image downloaded from:
# #https://pbs.twimg.com/media/C1MkrgQWQAASbdz.jpg
# # img=cv2.imread('images/HnE_Image.jpg', 1)
# # img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# # Io = 240 # Transmitted light intensity, Normalizing factor for image intensities
# # alpha = 1  #As recommend in the paper. tolerance for the pseudo-min and pseudo-max (default: 1)
# # beta = 0.15 #As recommended in the paper. OD threshold for transparent pixels (default: 0.15)


# def norm_HnE(img, Io=240, alpha=1, beta=0.15):


#     ######## Step 1: Convert RGB to OD ###################
#     ## reference H&E OD matrix.
#     #Can be updated if you know the best values for your image. 
#     #Otherwise use the following default values. 
#     #Read the above referenced papers on this topic. 
#     HERef = np.array([[0.5626, 0.2159],
#                       [0.7201, 0.8012],
#                       [0.4062, 0.5581]])
#     ### reference maximum stain concentrations for H&E
#     maxCRef = np.array([1.9705, 1.0308])
    
    
#     # extract the height, width and num of channels of image
#     h, w, c = img.shape
    
#     # reshape image to multiple rows and 3 columns.
#     #Num of rows depends on the image size (wxh)
#     img = img.reshape((-1,3))
    
#     # calculate optical density
#     # OD = −log10(I)  
#     #OD = -np.log10(img+0.004)  #Use this when reading images with skimage
#     #Adding 0.004 just to avoid log of zero. 
    
#     OD = -np.log10((img.astype(np.float)+1)/Io) #Use this for opencv imread
#     #Add 1 in case any pixels in the image have a value of 0 (log 0 is indeterminate)
    
    
#     ############ Step 2: Remove data with OD intensity less than β ############
#     # remove transparent pixels (clear region with no tissue)
#     ODhat = OD[~np.any(OD < beta, axis=1)] #Returns an array where OD values are above beta
#     #Check by printing ODhat.min()
    
#     ############# Step 3: Calculate SVD on the OD tuples ######################
#     #Estimate covariance matrix of ODhat (transposed)
#     # and then compute eigen values & eigenvectors.
#     eigvals, eigvecs = np.linalg.eigh(np.cov(ODhat.T))
    
    
#     ######## Step 4: Create plane from the SVD directions with two largest values ######
#     #project on the plane spanned by the eigenvectors corresponding to the two 
#     # largest eigenvalues    
#     That = ODhat.dot(eigvecs[:,1:3]) #Dot product
    
#     ############### Step 5: Project data onto the plane, and normalize to unit length ###########
#     ############## Step 6: Calculate angle of each point wrt the first SVD direction ########
#     #find the min and max vectors and project back to OD space
#     phi = np.arctan2(That[:,1],That[:,0])
    
#     minPhi = np.percentile(phi, alpha)
#     maxPhi = np.percentile(phi, 100-alpha)
    
#     vMin = eigvecs[:,1:3].dot(np.array([(np.cos(minPhi), np.sin(minPhi))]).T)
#     vMax = eigvecs[:,1:3].dot(np.array([(np.cos(maxPhi), np.sin(maxPhi))]).T)
    
    
#     # a heuristic to make the vector corresponding to hematoxylin first and the 
#     # one corresponding to eosin second
#     if vMin[0] > vMax[0]:    
#         HE = np.array((vMin[:,0], vMax[:,0])).T
        
#     else:
#         HE = np.array((vMax[:,0], vMin[:,0])).T
    
    
#     # rows correspond to channels (RGB), columns to OD values
#     Y = np.reshape(OD, (-1, 3)).T
    
#     # determine concentrations of the individual stains
#     C = np.linalg.lstsq(HE,Y, rcond=None)[0]
    
#     # normalize stain concentrations
#     maxC = np.array([np.percentile(C[0,:], 99), np.percentile(C[1,:],99)])
#     tmp = np.divide(maxC,maxCRef)
#     C2 = np.divide(C,tmp[:, np.newaxis])
    
#     ###### Step 8: Convert extreme values back to OD space
#     # recreate the normalized image using reference mixing matrix 
    
#     Inorm = np.multiply(Io, np.exp(-HERef.dot(C2)))
#     Inorm[Inorm>255] = 254
#     Inorm = np.reshape(Inorm.T, (h, w, 3)).astype(np.uint8)  
    
#     # Separating H and E components
#     H = np.multiply(Io, np.exp(np.expand_dims(-HERef[:,0], axis=1).dot(np.expand_dims(C2[0,:], axis=0))))
#     H[H>255] = 254
#     H = np.reshape(H.T, (h, w, 3)).astype(np.uint8)
    
#     E = np.multiply(Io, np.exp(np.expand_dims(-HERef[:,1], axis=1).dot(np.expand_dims(C2[1,:], axis=0))))
#     E[E>255] = 254
#     E = np.reshape(E.T, (h, w, 3)).astype(np.uint8)
    
#     return (Inorm, H, E)

In [ ]:
# #Extract a small region from the large file (level 0)
# #Let us extract a region from somewhere in the middle - coords 16k, 16k
# #Extract 1024,1024 region
# smaller_region = slide.read_region((10000,10000), 0, (5000,5000))
# smaller_region_RGB = smaller_region.convert('RGB')
# smaller_region_np = np.array(smaller_region_RGB)

# # plt.axis('off')
# # plt.imshow(smaller_region_np);

# norm_img, H_img, E_img = norm_HnE(smaller_region_np, Io=240, alpha=1, beta=0.15)

# plt.figure(figsize=(12, 12))
# plt.subplot(221)
# plt.title('Original Image')
# plt.imshow(smaller_region_np)
# plt.subplot(222)
# plt.title('Normalized Image')
# plt.imshow(norm_img)
# plt.subplot(223)
# plt.title('H image')
# plt.imshow(H_img)
# plt.subplot(224)
# plt.title('E image')
# plt.imshow(E_img)
# plt.show()